In [ ]:
import pickle
import xgboost
# import import_ipynb
from XGBoost import pre_process
import numpy as np
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, train_test_split
import pandas as pd
from sklearn.metrics import *
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import confusion_matrix

In [ ]:
file = open('pickle/model.pkl','rb')
added = pickle.load(file)

In [ ]:
added

In [ ]:
geog_lb = pickle.load(file)
# gender_lb = pickle.load(file)
# female_lb =  pickle.load(file)
# age_ss = pickle.load(file)
# tenure_ss = pickle.load(file)
# bal_ss = pickle.load(file)
# has_crd_ss = pickle.load(file)
# sal_ss=pickle.load(file)
# classifier = pickle.load(file)

In [ ]:
geog_lb

In [ ]:
lb_Geo = geog_lb[0]
lb_gen = geog_lb[1]
age_ss = geog_lb[2]
tenure_ss = geog_lb[3]
bal_ss = geog_lb[4]
has_crd_ss = geog_lb[5]
sal_ss= geog_lb[6]
classifier = geog_lb[7]

In [ ]:
test = pd.read_csv('Data/test.csv')

In [ ]:
X = test[['Age', 'Tenure', 'Geography','Gender','Balance', 'HasCrCard', 'EstimatedSalary']]
y = test['Exited']

In [ ]:
X.head()

In [ ]:
cols=['Geography','Gender']
added_cols =[]
def binarizer(X,col,lb):
    sorted_col = np.sort(X[col].unique())
    binarized = lb.transform(X[col])
    for i in range(0,len(sorted_col)-1):
        X[sorted_col[i]]=binarized[:,i]
        added_cols.append(sorted_col[i])
    X.drop([col],axis=1,inplace=True)
    return X

In [ ]:
X = binarizer(X,cols[0],lb_Geo)
X = binarizer(X,cols[1],lb_gen)

In [ ]:
def scaler(X, col, ss):
    if col not in added_cols:
        X[col] = ss.transform(np.array(X[col]).reshape(-1,1))

In [ ]:
added

In [ ]:
scaler(X,'Age',age_ss)
scaler(X,'Tenure',tenure_ss)
scaler(X,'Balance',bal_ss)
scaler(X,'HasCrCard',has_crd_ss)
scaler(X,'EstimatedSalary',sal_ss)

In [ ]:
X.head()

In [ ]:
pred = classifier.predict(X)

In [ ]:
confusion_matrix(y,pred)